<a href="https://colab.research.google.com/github/hungpham13/Laptop-EDA/blob/main/DS_Project_20211.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/hungpham13/Laptop-EDA.git

Cloning into 'Laptop-EDA'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 203 (delta 122), reused 96 (delta 43), pack-reused 0
Receiving objects: 100% (203/203), 12.29 MiB | 12.88 MiB/s, done.
Resolving deltas: 100% (122/122), done.


**[Guideline](https://husteduvn.sharepoint.com/sites/IntroductiontoDataScience-IT4142E-2021-2022/Class%20Materials/Capstone%20project/DS-Capstone%20project%20guidelines.pdf)**
## Topic description assignment (20th of October, 2021)
[Our project proposal](https://docs.google.com/document/d/16L1Mqy_HtDfhxB0xNXmCZSBNMKazWsYgwA9s1Vueih8/edit?usp=sharing)


## Assginment for each member
### **[On GG Sheet](https://docs.google.com/spreadsheets/d/11ldCYnPJwYuMKBba6jX7qPYnYtekAYjR9YvORww76hE/edit#)**

Distinguish between programming tasks and analytic tasks. 
- Examples of programming tasks: data scraping for source A; implementing algorithm A,
etc.
- Examples of analytic tasks: proposing our subject, selecting the algorithms to be used to
solve this problem, writing parts 1/2/3/4 of the report, writing parts 1/2/3/4 of the presentation, etc.

For each task, provide a detailed contribution of each member to this task:
- e.g.: Programming Task 1- data scraping from source A: member1 Mr X 40% ; member 2 Mrs Y 60% (no more than 2 members /task for programming tasks)...).
- e.g.: Analytic Task 1– proposing our subject: member1 Mr X 40% ; member 2 Mrs Y 30% ; member 2 Mr Z 30% (can be more than 2 members /task for analytic
tasks)...)

## Final assignment (29th of December, 2021)
    
Create 1 directory named with group number, upload on the dedicated assignment on Teams:
- Report: PDF file, no more than 15 pages (not included reference pages / first page / table of contents)
- Code: put in a single .zip file, with a readme.txt file explaining your codes structure, how to run the project – if necessary.
- Video demo of your Python code running in a notebook
- Presentation: PDF file, no more than 10 slides. The first slide MUST contain the (recent) pictures of the students in the group, with student’s name/number

# Materials

1.Scrapy, Selenium, BeautifulSoup –Data scraping
2.Numpy, Pandas, Seaborn –data analysis and visualization

[Kaggle detailed EDA](https://www.kaggle.com/ekami66/detailed-exploratory-data-analysis-with-python)

[Github DS project](https://github.com/veb-101/Data-Science-Projects)

[Havard Data Science Course](http://cs109.github.io/2015/pages/videos.html)

[Web Scraping with Python (Book)](https://vn1lib.org/book/3515980/5d50aa)

[Web Scraping on Kaggle](https://www.kaggle.com/getting-started?search=web+scraping)

In [2]:
import re
import pandas as pd
import numpy as np
# !pip install jellyfish
!pip install python-Levenshtein
!pip install fuzzywuzzy
from difflib import SequenceMatcher
# import jellyfish
from fuzzywuzzy import fuzz

     |████████████████████████████████| 50 kB 4.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149864 sha256=05f0e308e56f45ad272e11de8302b46759318e7f0398d0a53cb53860d8d717ad
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


# Data cleaning - Data preprocessing

In [3]:
def lower_all_string(df):
    '''lower all values in string columns of dataframe df'''
    for c in df.columns:
        if df[c].dtypes == np.object:
            df[c] = df[c].str.lower()

## Thegioididong - Dienmayxanh
result: no screen technology

In [4]:
def clean_thegioididong(source):
    source.dropna(subset=['ports'],inplace=True)
    result = pd.DataFrame()
    result['P/N'] = source['title'].str.extract(r"\(([^)]+)\)",expand=False).str.strip().str.replace("\.$", "") #replace any dot at the end of P/N

    result['Rating'] = source['rating'].astype('float64').fillna(0.0)
    result['Rating_count'] = source['review_count'].str.extract(r'(\d+)',expand=False).str.strip().fillna(0).astype('int64')

    result['Comment_count'] = 0
    for i,r in source['cmt_count'].iteritems():
        if pd.isnull(r):
            continue
        for c in r.split("|"):
            if re.search(r'([\d.,]+)',c):
                result['Comment_count'][i] += int(re.search(r'([\d.,]+)',c).group().replace(".",""))
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(result['Comment_count'].value_counts(dropna=False))

    result['Brand'] = source['title'].str.split().str[1].str.strip().str.lower()
    # print(result['Brand'].value_counts(dropna=False))
    result['Series'] = source['title'].str.extract(r'\w+\s\w+\s([\w\s]+)\s[\w\/d]+',expand=False).str.strip()
    result.loc[(result['Series'].str.split().str[-1].str.len() == 2), 'Series'] =  result['Series'].str.slice(stop=-3)
    # for a in zip(result.Series, source.title):
    #     print('{0:40} {1}'.format(a[0],a[1]))

    result['Price'] = source['price'].str.extract(r'([\d.]+)',expand=False).str.replace(".","").astype('float64')

    result['cpu_brand'] = source['cpu_name'].str.extract(r'([\w]+)\s',expand=False).str.strip()
    result['cpu_code'] = source['cpu_name'].str.extract(r'-([\w\d\s]+)',expand=False).str.strip()
    result['cpu_code'] = np.where(result['cpu_code'].str.len() > 10, 
                                    source['cpu_name'].str.extract(r'([\w\d\s]+)-',expand=False).str.split().str.slice(start=1).str.join(' '),
                                    result['cpu_code'])

    result['ram_size'] = source['ram_size'].str.extract(r'(\d+)',expand=False).str.strip().fillna(8).astype('float64')
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(result['ram_size'].value_counts(dropna=False))

    result['ram_upgradable'] = np.where(source['ram_upgradable'].str.contains("Kh"), "no", "yes")

    result['GPU'] = source['gpu'].str.extract(r'-([\w\d\s-]+),?',expand=False).str.strip()

    result['Battery'] = source['battery'].str.extract(r'([\d\s.]+)Wh',expand=False).str.strip().astype('float64')

    result['storage_size'] = np.nan
    result['storage_type'] = np.nan
    result['storage_extra_slot'] = np.nan
    # for i,r in result.loc[result['storage_size'].notnull(),'storage_size'].iteritems():
    for i,r in source['storage'].iteritems():
        if pd.isnull(r):
            continue
        for s in r.split("|"):
            s = s.lower()
            if "hỗ trợ" not in s:
                result['storage_size'][i] = re.search(r'(\d+)\s?[gtb]{2}',s.strip()).group(1)
                if "ssd" in s:
                    result['storage_type'][i] = "SSD"
                elif "hdd" in s:
                    result['storage_type'][i] = "HDD"
            elif "không" not in s: #just have "ho tro", no "khong"
                result['storage_extra_slot'][i] = 'yes'
        if pd.isnull(result['storage_extra_slot'][i]):
                result['storage_extra_slot'][i] = 'no'
    result['storage_size'] = np.where(result['storage_size']==1,1024,result['storage_size'])
    result['storage_size'] = np.where(result['storage_size']==2,2048,result['storage_size'])
    # print(result['storage_size'].value_counts(dropna=False))

    result['displ_size'] = source['displ_size'].str.extract(r'([\d.]+)',expand=False).astype('float64')
    result['displ_resolution'] = source['displ_resolution'].str.extract(r"\(([^)]+)\)",expand=False).str.strip()
    result['displ_rate'] = source['displ_rate'].str.extract(r'([\d]+)',expand=False).fillna(60).astype('int64')
    result['screen_technology'] = np.nan
    for i,r in source.displ_tech.iteritems():
        if pd.isnull(r):
            continue
        elif 'ips' in r.lower():
            result['screen_technology'][i] = 'IPS'
        elif 'oled' in r.lower():
            result['screen_technology'][i] = 'OLED'
        elif 'wva' in r.lower():
            result['screen_technology'][i] = 'VA'
    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     print(result['screen_technology'].value_counts(dropna=False))
    #     print(source.displ_tech[result.screen_technology.isnull()])

    result['Weight'] = source['weight_dims'].str.extract(r"Nặng ([\d.]+)",expand=False).str.strip().astype('float64')
    result['length'] = source['weight_dims'].str.extract(r"Dài ([\d.]+)",expand=False).str.strip().astype('float64')
    result['width'] = source['weight_dims'].str.extract(r"Rộng ([\d.]+)",expand=False).str.strip().astype('float64')
    result['thickness'] = source['weight_dims'].str.extract(r"Dày ([\d.]+)",expand=False).str.strip().astype('float64')

    result['OS'] = source['os'].str.split().str[:2].str.join(" ").str.strip()
    result['Surface'] = np.nan
    for i,r in source['surface'].iteritems():
        if pd.isnull(r):
            continue
        d = {"nhôm":"Aluminum","carbon":"Carbon","kim loại": "Metal", "nhựa":"Plastic","magie":"Magnesium"}
        s = []
        for vn_term in d:
            if vn_term in r.lower():
                s.append(d[vn_term])
        result['Surface'][i] = ', '.join(s)
        
    result['Release_year'] = source['launch']
    result['backlit_keyboard'] = np.where(source['backlight_key'].str.contains('Không'),'no','yes')

    result['hdmi'] = np.where(source['ports'].str.lower().str.contains("hdmi"),'yes','no')
    result['headphone_jack'] = np.where(source['ports'].str.lower().str.contains("jack"),'yes','no')
    result['lan'] = np.where(source['ports'].str.lower().str.contains("lan"),'yes','no')
    result['type_c'] = np.where(source['ports'].str.lower().str.contains("-c"),'yes','no')
    result['usb_a'] = np.where(source['ports'].str.lower().str.contains(r"usb [a32]",regex=True),'yes','no')
    result['sd_card'] = np.where(pd.isnull(source['sd_card']),'no','yes')
    lower_all_string(result)
    return result

In [5]:
scraped_tgdd = pd.read_csv("Laptop-EDA/scraped_data/tgdd_dmx.csv",index_col='url')
cleaned_tgdd = clean_thegioididong(scraped_tgdd)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(cleaned_tgdd.Series.value_counts(dropna=False))
#     print(cleaned_tgdd)
cleaned_tgdd.to_csv('Laptop-EDA/scraped_data/tgdd_dmx_cleaned.csv',encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: Set

## Fpt shop
- Rename column to the same format as data form
- Clean `Series`,`displ_rate`

In [6]:
scraped_fpt = pd.read_csv("Laptop-EDA/scraped_data/fptshop_scraped.csv")
cleaned_fpt = pd.read_csv("Laptop-EDA/scraped_data/fpt-cleaned.csv")
cleaned_fpt.rename(columns={'PN':'P/N',
                            'rating':'Rating',
                            'rating_count':'Rating_count',
                            'comment_count':'Comment_count',
                            'price':'Price',
                            'RAM':'ram_size',
                            'RAM_upgradable':'ram_upgradable',
                            'battery_wh':'Battery',
                            'storage_upgradable':'storage_extra_slot',
                            'screen_size':'displ_size',
                            'display_resolution':'displ_resolution',
                            'display_rating':'displ_rate',
                            'weight':'Weight',
                            'height':'thickness',
                            'surface_material':'Surface',
                            'release_year':'Release_year', 
                            'jack':'headphone_jack',
                            'usb':'usb_a', 
                            'sd':'sd_card',
                            'keyboard_light':'backlit_keyboard'
                            },inplace=True)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     scraped_fpt.info()
#     cleaned_fpt.info()
#     print(cleaned_fpt['Series'].value_counts(dropna=False))
for i,r in cleaned_fpt.Series.iteritems():
    extract_r1 = re.search(r'([\w\s-]+)[ /]c?o?r?e?\s?[ir\d]{2}[- ]',r.lower())
    extract_r2 = re.search(r'([\w\s-]+)pentium',r.lower())
    extract_r3 = re.search(r'([\w\s-]+)celeron',r.lower())
    extract_r4 = re.search(r'([\w\s-]+)n4020',r.lower())
    extract_r5 = re.search(r'([\w\s-]+)/n4500',r.lower())
    if extract_r1: extract_r = extract_r1.group(1).strip()
    elif extract_r2: extract_r = extract_r2.group(1).strip()
    elif extract_r3: extract_r = extract_r3.group(1).strip()
    elif extract_r4: extract_r = extract_r4.group(1).strip()
    elif extract_r5: extract_r = extract_r5.group(1).strip()
    else: extract_r = r.lower().strip()
    print('{0:55}  {1}'.format(r,extract_r))
    cleaned_fpt.Series[i] = extract_r

cleaned_fpt.displ_rate = cleaned_fpt.displ_rate.replace("--",60).fillna(60).astype('int64')
print(cleaned_fpt.displ_rate.value_counts(dropna=False))
lower_all_string(cleaned_fpt)
# cleaned_fpt[cleaned_fpt.Series == 'Flip BR1100FKA-BP0531T/N4500']

Nitro Gaming AN515-57-74NU/i7-11800H                     nitro gaming an515-57-74nu
Modern 14 B11MOU 852VN i5 1155G7/852VN                   modern 14 b11mou 852vn
Inspiron N3511 i3 1115G4/P112F001ABL                     inspiron n3511
Vostro V3500 i3 1115G4                                   vostro v3500
IdeaPad Slim 3 15ITL6 i3 1115G4                          ideapad slim 3 15itl6
Inspiron N3511 i5 1135G7/P112F001BBL                     inspiron n3511
Ideapad Slim 5 15ITL05 i5 1135G7                         ideapad slim 5 15itl05
Gaming GF63 Thin 10SC-481VN i7 10750H                    gaming gf63 thin 10sc-481vn
Inspiron N5515 R5 5500U/P106F003ASL                      inspiron n5515
ThinkBook 14 G3 ACL R7 5700U/21A2004BVN                  thinkbook 14 g3 acl
Vostro V3400 i5 1135G7/70253900                          vostro v3400
Vivobook A515EA-L11970T/i5-1135G7                        vivobook a515ea-l11970t
Nitro Gaming AN515-57-57MX/i5-11400H                     nitro gaming an515-57

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Cellphone S
- Drop column `'1star', '2star', '3star', '4star', '5star'`
- Rename column `comment_count`
- Change Price to float
- recleaned `Series`, `displ_size`

In [7]:
scraped_cellphones = pd.read_csv("Laptop-EDA/scraped_data/cellphones_scraped.csv",index_col='url')
# cleaned_cellphones2 = pd.read_csv("Laptop-EDA/scraped_data/cellphones_cleaned.csv",index_col='url')
cleaned_cellphones = pd.read_csv("Laptop-EDA/scraped_data/cellphones-cleaned.csv",index_col='url')

cleaned_cellphones.drop(columns=['1star', '2star', '3star', '4star', '5star'],inplace=True)
cleaned_cellphones.rename(columns={'comment_count':'Comment_count',
},inplace=True)
cleaned_cellphones['Price'] = cleaned_cellphones['Price'].str.extract(r'([\d.]+)',expand=False).str.replace(".","").astype('float64')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     scraped_cellphones.info()
#     cleaned_cellphones.info()
#     cleaned_cellphones2.info()
#     # print(scraped_cellphones['Series'].value_counts(dropna=False))
# cleaned_cellphones.head()

for i,cleaned_row in cleaned_cellphones.Series.iteritems():
    r = scraped_cellphones.item_name[i]
    if not isinstance(r, str):
        # print(r)
        r = r[0]
    black_key = [r'\d+gb',r'\d+inch',r'\d+gpu',r'\d+tb',
                 'laptop','chính','hãng','việt','nam','nhập','khẩu','cũ','đẹp',
                 'apple','asus','msi','hp','lenovo','dell','acer','lg','microsoft','huawei','avita','gigabyte','2020','2021']
    cpu_key = ['core','ryzen','m1']
    specs_key = [r'\d+\sgb',r'[\d.]+\sinch',r'\d+\sgpu',r'\d+\stb']
    extract_r1 = []
    for index,w in enumerate(r.lower().split()):
        if w in cpu_key:
            break
        if not any(bool(re.match(k,w)) for k in black_key) or (w == 'laptop' and index!=0):
            extract_r1.append(w)
    extract_r1 = ' '.join(extract_r1).strip(' -')
    for k in specs_key:
        extract_r1 = re.sub(k, '', extract_r1).strip(' -')

    # extract_r2 = re.search(r'([\w\s-]+)',r.lower())
    # print('{0:80} {1}'.format(r,extract_r1))
    print(f'{r:80} {cleaned_row:20} {extract_r1.strip()}')
    cleaned_cellphones.Series[i] = extract_r1.strip()

#recleaned displ_size
cleaned_cellphones.displ_size.loc['https://cellphones.com.vn/laptop-asus-rog-strix-g17-g713qr-hg073t.html'] = 17.3
cleaned_cellphones.displ_size.loc['https://cellphones.com.vn/laptop-hp-15-dy2093dx-405f7ua.html'] = 15.6
#lower all string attributes
lower_all_string(cleaned_cellphones)
# cleaned_cellphones.head()

Apple MacBook Air 13 inch 128GB MQD32 I Chính hãng Apple Việt Nam                MacBook Air          macbook air  mqd32 i
Laptop Dell Inspiron 3505 Y1N1T2                                                 Inspiron             inspiron 3505 y1n1t2
Huawei Matebook 14                                                               Matebook             matebook 14
Huawei Matebook X                                                                Matebook X           matebook x
iMac 24 2021 M1 7GPU 8GB 256GB I Chính hãng Apple Việt Nam                       iMac                 imac 24
Apple iMac 24 2021 M1 7GPU 16GB 256GB Chính Hãng                                 iMac                 imac 24
iMac 24 2021 M1 8GPU 8GB 256GB I Chính hãng Apple Việt Nam                       iMac                 imac 24
iMac 24 2021 M1 8GPU 16GB 256GB I Chính hãng Apple Việt Nam                      iMac                 imac 24
iMac 24 2021 M1 8GPU 16GB 512GB I Chính hãng Apple Việt Nam                      iMac  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Laptop Acer Aspire 5 A514-54-39KU NX.A23SV.003                                   Aspire               aspire 5 a514-54-39ku nx.a23sv.003
Laptop Acer Aspire 5 A514-54-51RB NX.A2ASV.003                                   Aspire               aspire 5 a514-54-51rb nx.a2asv.003
Laptop Acer Aspire 5 A514-54-540F NX.A28SV.005                                   Aspire               aspire 5 a514-54-540f nx.a28sv.005
Laptop Acer Aspire 5 A515-55-35SE                                                Aspire               aspire 5 a515-55-35se
Laptop Acer Aspire 5 A515-56-36UT                                                Aspire               aspire 5 a515-56-36ut
Laptop Acer Aspire 5 A515-56-54PK NX.A1GSV.002                                   Aspire               aspire 5 a515-56-54pk nx.a1gsv.002
Laptop Acer Asipre 5 A515-56G-51YL NX.A1LSV.002                                  Aspire               asipre 5 a515-56g-51yl nx.a1lsv.002
Laptop Acer Gaming Aspire 7 A715-42G-R1SB NH.QAYSV.005            

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


# Integrate
cleaned data are saved as `cleaned_tgdd`,`cleaned_fpt`,`cleaned_cellphones`

TODO
- Chọn các attribute và xử lý các attribute đó trong cả 3 file - Series ✅
- Kết nối 3 data của tgdd, cellphone và fpt bao gồm các mapping attribute và hợp nhất các retained attribute
- Kết nối data thu được ở bước 2 với data của laptop arena dựa vào các mapping attribute, giữ lại các retained.
- Xử lý data đã map được ở bước 3 (loại bỏ các cột quá nhiều null, format dữ liệu, điền tay cho các máy của apple)


## Check out mapping attribute in TGDD, FPT and CellphoneS
- Mapping attribute must be non-null, except for Apple brand ✅
- Retained attribute type is float or int ✅
- Three dataset must have same set of column ✅
- No duplicated rows

In [8]:
cleaned_fpt[['Series','Brand','cpu_brand','cpu_code','ram_size','storage_size',
             'displ_size','displ_rate',
             'Price','Rating','Rating_count','Comment_count']].info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(cleaned_fpt.Brand[cleaned_fpt.cpu_code.isnull()])
    print(cleaned_fpt[cleaned_fpt['P/N'].isnull()])
cleaned_fpt.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series         146 non-null    object 
 1   Brand          146 non-null    object 
 2   cpu_brand      146 non-null    object 
 3   cpu_code       145 non-null    object 
 4   ram_size       146 non-null    int64  
 5   storage_size   146 non-null    int64  
 6   displ_size     146 non-null    float64
 7   displ_rate     146 non-null    int64  
 8   Price          146 non-null    int64  
 9   Rating         143 non-null    float64
 10  Rating_count   143 non-null    float64
 11  Comment_count  143 non-null    float64
dtypes: float64(4), int64(4), object(4)
memory usage: 13.8+ KB
66    apple
Name: Brand, dtype: object
    P/N  Rating  Rating_count  Comment_count Brand  \
67  NaN     NaN           NaN            NaN   msi   

                            Series     Price cpu_brand cpu_code  ram

,P/N,Rating,Rating_count,Comment_count,Brand,Series,Price,cpu_brand,cpu_code,ram_size,ram_upgradable,GPU,Battery,storage_type,storage_size,storage_extra_slot,displ_size,displ_resolution,displ_rate,screen_technology,Weight,length,width,thickness,OS,Surface,Release_year,hdmi,headphone_jack,lan,type_c,usb_a,sd_card,backlit_keyboard
0,nh.qd9sv.001,5.0,1.0,598.0,acer,nitro gaming an515-57-74nu,28499000,intel,11800h,8,yes,nvidia geforce rtx 3050ti 4gb,--,ssd,512,yes,15.6,1920 x 1080,144,led-backlit,2.2,363.4,255,23.9,windows 10,nhựa,2021,yes,yes,yes,yes,yes,no,yes
1,852vn,5.0,1.0,340.0,msi,modern 14 b11mou 852vn,18799000,intel,1155g7,8,yes,intel iris xe graphics,65 w,ssd,512,yes,14.0,1920 x 1080,60,ips lcd,1.3,319,219,16.9,windows 10,kim loại,2021,yes,yes,yes,yes,yes,yes,yes
2,p112f001,5.0,1.0,116.0,dell,inspiron n3511,15299000,intel,1115g4,4,yes,intel uhd graphics,65 w,ssd,256,yes,15.6,1920 x 1080,60,wva anti-glare led backlit narrow border,1.73,358.5,258,18.9,windows 10,nhựa,2021,yes,yes,yes,yes,yes,yes,yes


In [9]:
cleaned_cellphones[['P/N','Series','Brand','cpu_brand','cpu_code','ram_size','storage_size','displ_size','displ_rate','Price','Rating','Rating_count','Comment_count']].info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(cleaned_cellphones.Brand[cleaned_cellphones.cpu_code.isnull()])
    print(cleaned_cellphones.Brand[cleaned_cellphones.ram_size.isnull()])
    print(cleaned_cellphones.Brand[cleaned_cellphones.storage_size.isnull()])
    print(cleaned_cellphones.Brand[cleaned_cellphones.displ_size.isnull()])
cleaned_cellphones.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 423 entries, https://cellphones.com.vn/apple-macbook-air-13-inch-128-gb-mqd32-chinh-hang.html to https://cellphones.com.vn/surface-pro-7-plus.html
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   P/N            357 non-null    object 
 1   Series         423 non-null    object 
 2   Brand          423 non-null    object 
 3   cpu_brand      423 non-null    object 
 4   cpu_code       401 non-null    object 
 5   ram_size       422 non-null    float64
 6   storage_size   422 non-null    float64
 7   displ_size     418 non-null    float64
 8   displ_rate     423 non-null    int64  
 9   Price          357 non-null    float64
 10  Rating         81 non-null     float64
 11  Rating_count   423 non-null    int64  
 12  Comment_count  423 non-null    int64  
dtypes: float64(5), int64(3), object(5)
memory usage: 66.3+ KB
url
https://cellphones.com.vn/apple-macbook-air-13-i

,sd_card,P/N,ram_upgradable,GPU,displ_rate,backlit_keyboard,Weight,thickness,width,length,screen_technology,Battery,OS,displ_resolution,Surface,displ_size,Price,Comment_count,Rating_count,Rating,ram_size,cpu_code,cpu_brand,storage_size,storage_type,storage_extra_slot,Release_year,Brand,Series,usb_a,type_c,lan,headphone_jack,hdmi
url,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
https://cellphones.com.vn/apple-macbook-air-13-inch-128-gb-mqd32-chinh-hang.html,yes,mqd32,NaN,intel hd 6000,60,NaN,1.35,17,227.0,325.00,NaN,54.0,macos,1440 x 900,NaN,13.3,20000000.0,306,0,NaN,8.0,NaN,intel,128.0,ssd,NaN,2017.0,apple,macbook air mqd32 i,yes,no,no,no,no
https://cellphones.com.vn/aptop-dell-inspiron-15-3505-y1n1t2.html,no,y1n1t2,yes,amd radeon,60,no,1.83,18,249.0,363.96,NaN,42.0,windows 10,1920 x 1080,NaN,15.6,18290000.0,28,0,NaN,8.0,3500u,amd,512.0,ssd,NaN,NaN,dell,inspiron 3505 y1n1t2,yes,no,yes,yes,yes
https://cellphones.com.vn/huawei-matebook-14.html,no,NaN,NaN,nvidia geforce mx350,60,NaN,1.53,15.9,223.8,307.50,NaN,56.0,windows 10,2160 x 1440,NaN,14.0,NaN,3,0,NaN,8.0,10210u,intel,512.0,ssd,NaN,NaN,huawei,matebook 14,no,yes,no,yes,yes


In [10]:
cleaned_tgdd[['P/N','Series','Brand','cpu_brand','cpu_code','ram_size','storage_size','displ_size','displ_rate','Price','Rating','Rating_count','Comment_count']].info()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(cleaned_tgdd.Brand[cleaned_tgdd.cpu_code.isnull()])
cleaned_tgdd.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 181 entries, https://www.thegioididong.com/laptop/msi-gf66-11uc-i7-224vn?src=osp#top-tskt to https://www.thegioididong.com/laptop/acer-aspire-a315-56-308n-i3-nxhs5sv00c?src=osp#top-tskt
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   P/N            181 non-null    object 
 1   Series         181 non-null    object 
 2   Brand          181 non-null    object 
 3   cpu_brand      181 non-null    object 
 4   cpu_code       174 non-null    object 
 5   ram_size       181 non-null    float64
 6   storage_size   181 non-null    float64
 7   displ_size     181 non-null    float64
 8   displ_rate     181 non-null    int64  
 9   Price          181 non-null    float64
 10  Rating         181 non-null    float64
 11  Rating_count   181 non-null    int64  
 12  Comment_count  181 non-null    int64  
dtypes: float64(5), int64(3), object(5)
memory usage: 24.8+ KB
url
https://w

,P/N,Rating,Rating_count,Comment_count,Brand,Series,Price,cpu_brand,cpu_code,ram_size,ram_upgradable,GPU,Battery,storage_size,storage_type,storage_extra_slot,displ_size,displ_resolution,displ_rate,screen_technology,Weight,length,width,thickness,OS,Surface,Release_year,backlit_keyboard,hdmi,headphone_jack,lan,type_c,usb_a,sd_card
url,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
https://www.thegioididong.com/laptop/msi-gf66-11uc-i7-224vn?src=osp#top-tskt,224vn,3.0,5,70,msi,katana gaming gf66 11uc,29990000.0,intel,11800h,8.0,yes,nvidia geforce rtx3050,53.0,512.0,ssd,no,15.6,1920 x 1080,144,ips,2.10,359.0,259.0,24.90,windows 10,plastic,2021.0,yes,yes,yes,yes,yes,yes,no
https://www.thegioididong.com/laptop/msi-gaming-pulse-gl66-11udk-i7-816vn?src=osp#top-tskt,816vn,0.0,0,13,msi,gaming pulse gl66 11udk,33990000.0,intel,11800h,16.0,yes,nvidia geforce rtx3050ti,53.5,512.0,ssd,no,15.6,1920 x 1080,144,ips,2.25,359.0,259.0,23.95,windows 10,metal,2021.0,yes,yes,no,yes,yes,yes,no
https://www.thegioididong.com/laptop/acer-nitro-gaming-an515-57-727j-i7-nhqd9sv005?src=osp#top-tskt,nh.qd9sv.005,3.6,13,220,acer,nitro 5 gaming an515 57 727j,29990000.0,intel,11800h,8.0,yes,nvidia geforce rtx3050ti,57.0,512.0,ssd,yes,15.6,1920 x 1080,144,ips,2.20,363.4,255.0,23.90,windows 10,plastic,2021.0,yes,yes,yes,yes,yes,yes,no


In [11]:
print(sorted(cleaned_fpt.columns))
print(sorted(cleaned_tgdd.columns))
print(sorted(cleaned_cellphones.columns))

['Battery', 'Brand', 'Comment_count', 'GPU', 'OS', 'P/N', 'Price', 'Rating', 'Rating_count', 'Release_year', 'Series', 'Surface', 'Weight', 'backlit_keyboard', 'cpu_brand', 'cpu_code', 'displ_rate', 'displ_resolution', 'displ_size', 'hdmi', 'headphone_jack', 'lan', 'length', 'ram_size', 'ram_upgradable', 'screen_technology', 'sd_card', 'storage_extra_slot', 'storage_size', 'storage_type', 'thickness', 'type_c', 'usb_a', 'width']
['Battery', 'Brand', 'Comment_count', 'GPU', 'OS', 'P/N', 'Price', 'Rating', 'Rating_count', 'Release_year', 'Series', 'Surface', 'Weight', 'backlit_keyboard', 'cpu_brand', 'cpu_code', 'displ_rate', 'displ_resolution', 'displ_size', 'hdmi', 'headphone_jack', 'lan', 'length', 'ram_size', 'ram_upgradable', 'screen_technology', 'sd_card', 'storage_extra_slot', 'storage_size', 'storage_type', 'thickness', 'type_c', 'usb_a', 'width']
['Battery', 'Brand', 'Comment_count', 'GPU', 'OS', 'P/N', 'Price', 'Rating', 'Rating_count', 'Release_year', 'Series', 'Surface', 'Wei

In [12]:
print(f"Duplicated tgdd: {any(cleaned_tgdd.duplicated())}")
print(f"Duplicated cellphones: {any(cleaned_cellphones.duplicated())}")
print(f"Duplicated fpt: {any(cleaned_fpt.duplicated())}")

Duplicated tgdd: False
Duplicated cellphones: False
Duplicated fpt: False


## Merge TGDD, FPT and CellphoneS
Retained attributes:
- Comment = Average of 3
- Rating Count = Sum of 3
- Rating point = Sum of Rating_count*Rating
- Price = Average of 3

String similarity
<ol>
<li><a href="http://en.wikipedia.org/wiki/Hamming_distance">Hamming distance</a>  </li>
<li><a href="http://en.wikipedia.org/wiki/Levenshtein_distance">Levenshtein distance</a></li>
<li><a href="http://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance">Damerau–Levenshtein distance</a>  </li>
<li><a href="http://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance">Jaro–Winkler distance</a></li>
</ol>

Jellyfish: https://github.com/jamesturk/jellyfish

In [13]:

def similar_str(a, b, algo):
    '''
    Compare string similarity with threshold
    Input: a,b - string, thres - float 
    Output: float
    '''
    if algo == 'normal':
        return SequenceMatcher(None, a, b).ratio()
    elif algo == 'lev':
        return fuzz.ratio(a,b)/100

s1 = 'gaming predator helios 300 ph315-54-78w5 nh.qc5sv.001'
s2 = 'Predator Helios 300 PH315-53-781R NH.Q7YAA.001'.lower()
s3 = '15.2'
s4 = '14.2'
print(similar_str(s3,s4,'lev'))
print(similar_str(s2,s1,'lev'))

0.75
0.79


In [35]:
# r1 = cleaned_fpt.iloc[134]
# print(r1)
# min_score = 0
# algo = 'lev'
# choices = []
# for i,r2 in cleaned_tgdd.iterrows():
#     score = similar_str(r1['Series'],r2['Series'],algo)
#     condition = [r1['ram_size'] == r2['ram_size'],
#                     r1['cpu_brand'] == r2['cpu_brand'],
#                     r1['cpu_code'] == r2['cpu_code'],
#                     r1['Brand'] == r2['Brand'],
#                     r1['storage_size'] == r2['storage_size'],
#                     score > min_score
#     ]
#     if all(condition):
#         choices.append((i,score))
# print(choices)
# choice = max(choices, key=lambda x:x[1])[0]
# for k,v1,v2 in zip(cleaned_fpt.columns,r1, cleaned_fpt.loc[choice]):
#     print(f"{k:40} {v1:40} {v2}")

def merge(dfa,dfb,pref1='_df1',pref2='_df2'):
    min_score = 0.6
    algo = 'lev'

    df1 = dfa.copy()
    df2 = dfb.copy()

    df1['index'+pref1]=df1.index
    df2['index'+pref2]=df2.index

    df1['df2_matchindex'] = np.nan
    df1['match_score'] = np.nan
    
    choices = []
    for i1,r1 in df1.iterrows():
        for i2,r2 in df2.iterrows():
            score = similar_str(r1['Series'],r2['Series'],algo)
            condition = [r1['ram_size'] == r2['ram_size'],
                         r1['cpu_brand'] == r2['cpu_brand'],
                         r1['cpu_code'] == r2['cpu_code'],
                         r1['Brand'] == r2['Brand'],
                         r1['storage_size'] == r2['storage_size'],
                         r1['displ_rate'] == r2['displ_rate'],
                         abs(r1['displ_size']-r2['displ_size'])<0.5,
                         score > min_score
            ]
            if all(condition):
                choices.append([i1,i2,score])

    exclude1, exclude2 = [], []
    for i1,i2,score in sorted(choices,key=lambda x:x[2],reverse=True):
        if i1 not in exclude1 and i2 not in exclude2:
            df1['df2_matchindex'][i1] = i2
            df1['match_score'][i1] = score
            exclude1.append(i1)
            exclude2.append(i2)
    
    print(f"Matched: {len(df1['df2_matchindex'][np.invert(df1['df2_matchindex'].isnull())])}")
    merged = pd.merge(df1, df2,
                  left_on='df2_matchindex',
                  right_index = True,
                  suffixes=[pref1,pref2],
                  how = 'outer') 
    return merged.reindex(sorted(merged.columns), axis=1)

In [36]:
df = merge(cleaned_fpt, cleaned_tgdd,'_fpt','_tgdd')
# df = merge(cleaned_tgdd, cleaned_fpt,'_tgdd','_fpt')
df.head()

Matched: 61


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Battery_fpt,Battery_tgdd,Brand_fpt,Brand_tgdd,Comment_count_fpt,Comment_count_tgdd,GPU_fpt,GPU_tgdd,OS_fpt,OS_tgdd,P/N_fpt,P/N_tgdd,Price_fpt,Price_tgdd,Rating_count_fpt,Rating_count_tgdd,Rating_fpt,Rating_tgdd,Release_year_fpt,Release_year_tgdd,Series_fpt,Series_tgdd,Surface_fpt,Surface_tgdd,Weight_fpt,Weight_tgdd,backlit_keyboard_fpt,backlit_keyboard_tgdd,cpu_brand_fpt,cpu_brand_tgdd,cpu_code_fpt,cpu_code_tgdd,df2_matchindex,displ_rate_fpt,displ_rate_tgdd,displ_resolution_fpt,displ_resolution_tgdd,displ_size_fpt,displ_size_tgdd,hdmi_fpt,hdmi_tgdd,headphone_jack_fpt,headphone_jack_tgdd,index_fpt,index_tgdd,lan_fpt,lan_tgdd,length_fpt,length_tgdd,match_score,ram_size_fpt,ram_size_tgdd,ram_upgradable_fpt,ram_upgradable_tgdd,screen_technology_fpt,screen_technology_tgdd,sd_card_fpt,sd_card_tgdd,storage_extra_slot_fpt,storage_extra_slot_tgdd,storage_size_fpt,storage_size_tgdd,storage_type_fpt,storage_type_tgdd,thickness_fpt,thickness_tgdd,type_c_fpt,type_c_tgdd,usb_a_fpt,usb_a_tgdd,width_fpt,width_tgdd
0.0,--,57.0,acer,acer,598.0,264.0,nvidia geforce rtx 3050ti 4gb,nvidia geforce rtx3050ti,windows 10,windows 10,nh.qd9sv.001,nh.qd9sv.001,28499000.0,29990000.0,1.0,0.0,5.0,0.0,2021,2021.0,nitro gaming an515-57-74nu,nitro 5 gaming an515 57 74nu,nhựa,plastic,2.2,2.2,yes,yes,intel,intel,11800h,11800h,https://www.thegioididong.com/laptop/acer-nitr...,144.0,144.0,1920 x 1080,1920 x 1080,15.6,15.6,yes,yes,yes,yes,0.0,https://www.thegioididong.com/laptop/acer-nitr...,yes,yes,363.4,363.4,0.89,8.0,8.0,yes,yes,led-backlit,ips,no,no,yes,yes,512.0,512.0,ssd,ssd,23.9,23.9,yes,yes,yes,yes,255,255.0
1.0,65 w,39.0,msi,msi,340.0,53.0,intel iris xe graphics,nvidia geforce mx450,windows 10,windows 10,852vn,669vn,18799000.0,22990000.0,1.0,0.0,5.0,0.0,2021,2021.0,modern 14 b11mou 852vn,gaming modern 14 b11sbu,kim loại,metal,1.3,1.3,yes,yes,intel,intel,1155g7,1155g7,https://www.thegioididong.com/laptop/msi-gamin...,60.0,60.0,1920 x 1080,1920 x 1080,14.0,14.0,yes,yes,yes,yes,1.0,https://www.thegioididong.com/laptop/msi-gamin...,yes,no,319,319.0,0.62,8.0,8.0,yes,yes,ips lcd,ips,yes,yes,yes,no,512.0,512.0,ssd,ssd,16.9,16.9,yes,yes,yes,yes,219,219.0
2.0,65 w,41.0,dell,dell,116.0,206.0,intel uhd graphics,intel uhd graphics,windows 10,windows 10,p112f001,p112f001abl,15299000.0,15290000.0,1.0,18.0,5.0,3.7,2021,2021.0,inspiron n3511,inspiron 15 3511,nhựa,plastic,1.73,1.7,yes,no,intel,intel,1115g4,1115g4,https://www.thegioididong.com/laptop/dell-insp...,60.0,60.0,1920 x 1080,1920 x 1080,15.6,15.6,yes,yes,yes,yes,2.0,https://www.thegioididong.com/laptop/dell-insp...,yes,no,358.5,358.5,0.87,4.0,4.0,yes,yes,wva anti-glare led backlit narrow border,va,yes,yes,yes,yes,256.0,256.0,ssd,ssd,18.9,18.9,yes,no,yes,yes,258,235.5
3.0,45 w,NaN,dell,NaN,788.0,NaN,intel uhd graphics,NaN,windows 10,NaN,v5i3001w,NaN,15599000.0,NaN,2.0,NaN,5.0,NaN,2021,NaN,vostro v3500,NaN,nhựa,NaN,1.98,NaN,yes,NaN,intel,NaN,1115g4,NaN,NaN,60.0,NaN,1920 x 1080,NaN,15.6,NaN,yes,NaN,yes,NaN,3.0,NaN,yes,NaN,368,NaN,NaN,8.0,NaN,yes,NaN,anti-glare led-backlit,NaN,yes,NaN,yes,NaN,256.0,NaN,ssd,NaN,19.9,NaN,yes,NaN,yes,NaN,249,NaN
4.0,--,NaN,lenovo,NaN,378.0,NaN,intel uhd graphics,NaN,windows 10,NaN,82h80040vn,NaN,14999000.0,NaN,4.0,NaN,5.0,NaN,2021,NaN,ideapad slim 3 15itl6,NaN,abs plastic,NaN,1.65,NaN,yes,NaN,intel,NaN,1115g4,NaN,NaN,60.0,NaN,1920 x 1080,NaN,15.6,NaN,yes,NaN,yes,NaN,4.0,NaN,yes,NaN,359,NaN,NaN,8.0,NaN,yes,NaN,anti-glare led-backlit,NaN,yes,NaN,yes,NaN,512.0,NaN,ssd,NaN,19.9,NaN,yes,NaN,yes,NaN,237.7,NaN


In [38]:
def print_merged_result(df, col='all',pref1='_fpt',pref2='_tgdd'):
    def print_col(df,col):
        print(f"-------Column: {col} ---------------")
        for i, r in df.iterrows():
            # if abs(r[col+pref1]-r[col+pref2]) >=0.5:
            print(f"{r['match_score']:4}   {r[col+pref1]:40} {r[col+pref2]:40} {r['index'+pref1]:10} {r['index'+pref2]}")
    
    all_cols = ['Battery', 'Brand', 'Comment_count', 'GPU', 'OS', 'P/N', 
                'Price', 'Rating', 'Rating_count', 'Release_year', 'Series', 
                'Surface', 'Weight', 'backlit_keyboard', 'cpu_brand', 'cpu_code', 
                'displ_rate', 'displ_resolution', 'displ_size', 'hdmi', 'headphone_jack', 
                'lan', 'length', 'ram_size', 'ram_upgradable', 'screen_technology',
                'sd_card', 'storage_extra_slot', 'storage_size', 'storage_type', 
                'thickness', 'type_c', 'usb_a', 'width']
            
    if col=='all':
        for c in all_cols:
            print_col(df,c)
    else:
        print_col(df,col)
print_merged_result(df,col='Series')
#     print(cleaned_tgdd.Series.value_counts(dropna=False))

-------Column: Series ---------------
0.89   nitro gaming an515-57-74nu               nitro 5 gaming an515 57 74nu                    0.0 https://www.thegioididong.com/laptop/acer-nitro-5-gaming-an515-57-74nu-i7-nhqd9sv001?src=osp#top-tskt
0.62   modern 14 b11mou 852vn                   gaming modern 14 b11sbu                         1.0 https://www.thegioididong.com/laptop/msi-gaming-modern-14-b11sbu-i5-669vn?src=osp#top-tskt
0.87   inspiron n3511                           inspiron 15 3511                                2.0 https://www.thegioididong.com/laptop/dell-inspiron-15-3511-i3-p112f001abl?src=osp#top-tskt
 nan   vostro v3500                                                                  nan        3.0 nan
 nan   ideapad slim 3 15itl6                                                         nan        4.0 nan
 nan   ideapad slim 5 15itl05                                                        nan        6.0 nan
 nan   inspiron n5515                                             

In [34]:
r1 = cleaned_fpt.iloc[26]
print(r1)

P/N                                       82a30071vn
Rating                                             2
Rating_count                                       3
Comment_count                                    421
Brand                                         lenovo
Series                           yoga slim 7 14itl05
Price                                       23499000
cpu_brand                                      intel
cpu_code                                      1135g7
ram_size                                          16
ram_upgradable                                   yes
GPU                           intel iris xe graphics
Battery                                         65 w
storage_type                                     ssd
storage_size                                     512
storage_extra_slot                               yes
displ_size                                        14
displ_resolution                         1920 x 1080
displ_rate                                    

In [17]:
df = cleaned_tgdd.merge(cleaned_cellphones,on = 'P/N', how='inner')
df.reindex(sorted(df.columns), axis=1)

,Battery_x,Battery_y,Brand_x,Brand_y,Comment_count_x,Comment_count_y,GPU_x,GPU_y,OS_x,OS_y,P/N,Price_x,Price_y,Rating_count_x,Rating_count_y,Rating_x,Rating_y,Release_year_x,Release_year_y,Series_x,Series_y,Surface_x,Surface_y,Weight_x,Weight_y,backlit_keyboard_x,backlit_keyboard_y,cpu_brand_x,cpu_brand_y,cpu_code_x,cpu_code_y,displ_rate_x,displ_rate_y,displ_resolution_x,displ_resolution_y,displ_size_x,displ_size_y,hdmi_x,hdmi_y,headphone_jack_x,headphone_jack_y,lan_x,lan_y,length_x,length_y,ram_size_x,ram_size_y,ram_upgradable_x,ram_upgradable_y,screen_technology_x,screen_technology_y,sd_card_x,sd_card_y,storage_extra_slot_x,storage_extra_slot_y,storage_size_x,storage_size_y,storage_type_x,storage_type_y,thickness_x,thickness_y,type_c_x,type_c_y,usb_a_x,usb_a_y,width_x,width_y
0,57.5,57.0,acer,acer,119,3,nvidia geforce rtx 3060,nvidia geforce rtx 3060,windows 10,windows 10,nh.qdgsv.003,32990000.0,NaN,0,0,0.0,NaN,2021.0,NaN,nitro 5 gaming an515 57 5831,gaming nitro 5 eagle an515-57-5831 nh.qdgsv.003,plastic,plastic,2.200,2.200,yes,yes,intel,intel,11400h,11400h,144,144,1920 x 1080,1920 x 1080,15.6,15.6,yes,yes,yes,yes,yes,no,363.40,363.40,8.0,8.0,yes,yes,ips,ips,no,yes,yes,NaN,512.0,512.0,ssd,ssd,23.9,23.9,yes,yes,yes,yes,255.00,255.00
1,48.0,48.0,acer,acer,375,6,nvidia geforce gtx 1650 4 gb,nvidia geforce gtx 1650,windows 10,windows 10,nh.qaysv.003,22490000.0,21190000.0,23,0,4.8,NaN,2021.0,NaN,aspire 7 gaming a715 42g r6zr,gaming aspire 7 a715-42g-r6zr nh.qaysv.003,plastic,plastic,2.100,2.100,yes,yes,amd,amd,5500u,5500u,144,144,1920 x 1080,1920 x 1080,15.6,15.6,yes,yes,yes,yes,yes,yes,363.40,363.40,8.0,8.0,yes,yes,ips,ips,no,yes,no,no,512.0,512.0,ssd,ssd,22.9,22.9,yes,yes,yes,yes,254.50,254.50
2,67.0,67.0,asus,asus,9,0,intel iris xe graphics,intel iris xe,windows 10,windows 10,hl494ts,41990000.0,39990000.0,0,0,0.0,NaN,2021.0,NaN,zenbook ux371ea,zenbook ux371ea,metal,plastic,1.200,1.200,yes,yes,intel,intel,1165g7,1165g7,60,60,3840 x 2160,3840 x 2160,13.3,13.3,yes,yes,no,no,no,no,305.00,305.00,16.0,16.0,no,no,ips,oled,no,no,no,NaN,1024.0,1024.0,ssd,ssd,13.9,13.9,yes,yes,yes,yes,211.00,211.00
3,70.0,70.0,msi,msi,28,0,intel iris xe graphics,intel iris xe,windows 10,windows 10,211vn,39990000.0,37990000.0,0,0,0.0,NaN,2021.0,NaN,summit e13 flip,summit e13 flip evo a11mt-211vn,metal,NaN,1.350,1.350,yes,yes,intel,intel,1185g7,1185g7,60,60,1920 x 1200,1920 x 1200,13.4,13.4,no,no,yes,yes,no,no,300.20,300.20,16.0,16.0,no,no,ips,ips,yes,yes,no,NaN,1024.0,1024.0,ssd,ssd,14.9,14.9,yes,yes,yes,yes,222.25,222.25
4,50.0,50.0,lenovo,lenovo,81,24,intel iris xe graphics,intel iris xe,windows 10,windows 10,82ev0017vn,32740000.0,31660000.0,6,0,3.0,NaN,2020.0,NaN,yoga slim 7 carbon 13itl5,yoga slim 7i carbon 13itl5,"aluminum, carbon, magnesium",NaN,0.966,0.000,yes,yes,intel,intel,1165g7,1165g7,60,60,2560 x 1600,2560 x 1600,13.3,13.3,no,no,yes,no,no,no,295.88,295.88,16.0,16.0,no,no,ips,ips,no,no,no,NaN,1024.0,1024.0,ssd,ssd,15.0,14.25,yes,yes,no,no,208.85,208.85
5,57.0,59.0,acer,acer,181,5,nvidia geforce rtx3050ti,nvidia geforce rtx 3050ti,windows 10,windows 10,nh.qc5sv.001,33990000.0,NaN,1,0,2.0,NaN,2021.0,NaN,predator helios ph315 54 78w5,gaming predator helios 300 ph315-54-78w5 nh.qc...,"metal, plastic",NaN,2.300,2.300,yes,yes,intel,intel,11800h,11800h,144,144,1920 x 1080,1920 x 1080,15.6,15.6,yes,yes,yes,yes,yes,yes,363.00,363.00,8.0,8.0,yes,yes,ips,ips,no,yes,yes,NaN,512.0,512.0,ssd,ssd,22.9,22.9,yes,yes,yes,yes,255.00,255.00
6,67.0,67.0,asus,asus,48,0,intel iris xe graphics,intel iris xe,windows 10,windows 10,hp163t,33990000.0,32590000.0,0,0,0.0,NaN,2021.0,NaN,zenbook flip ux363ea,zenbook flip ux363ea,metal,aluminum,1.300,1.300,yes,yes,intel,intel,1165g7,1165g7,60,60,1920 x 1080,1920 x 1080,13.3,13.3,yes,yes,no,no,no,no,305.00,304.20,16.0,16.0,no,no,oled,oled,no,yes,no,NaN,512.0,512.0,ssd,ssd,11.9,13.9,yes,yes,yes,yes,211.00,203.00
7,51.0,NaN,msi,msi,84,19,q,nvidia geforce rtx 3060,windows 10,windows 10,228vn,32490000.0,30590000.0,4,0,4.5,NaN,20